In [4]:
import pandas as pd
import os

In [5]:
current_dir = os.getcwd()
results = pd.read_csv(current_dir + '/results.csv')

In [6]:
'''MISA-ConfidNet results
columns
    id: id of the video
    input_sentence: actual text input
    label: ground truth label list
    prediction: predicted label list
    confidence: confidence of the prediction
    confidence-t: confience while removing the text modality
    confidence-a: confience while removing the audio modality
    confidence-v: confience while removing the video modality
labels
    0: happiness
    1: sadness
    2: anger
    3: fear
    4: disgust
    5: surprise
'''
results.head()

,id,input_sentence,label,prediction,confidence,confidence-t,confidence-v,confidence-a
0,-6rXp3zJ3kc[0],future and it's a retirement future that can u...,[0. 1. 1. 0. 0. 0.],[1. 1. 0. 0. 0. 0.],0.284847,0.185451,-1.661398,0.315847
1,-6rXp3zJ3kc[1],very aware of many people are not until they g...,[0. 1. 0. 0. 0. 0.],[1. 1. 1. 0. 0. 0.],0.280548,0.161487,-1.413134,0.311320
2,-6rXp3zJ3kc[2],so the answer to the question can i contribute...,[0. 1. 0. 0. 0. 0.],[0. 1. 0. 0. 0. 1.],0.323583,0.216484,5.152101,-0.177674
3,-9y-fZ3swSY[0],is you can say hey i really like baby skin the...,[1. 0. 0. 0. 0. 0.],[1. 0. 0. 0. 0. 0.],0.277032,0.242510,0.247198,0.265411
4,-9y-fZ3swSY[1],face so nice by giving a good example and not ...,[1. 0. 0. 0. 0. 0.],[1. 0. 1. 0. 1. 0.],0.302093,0.215045,0.315789,0.360302


In [11]:
text_is_noise = results[results['confidence-t'] > results['confidence']]
audio_is_noise = results[results['confidence-a'] > results['confidence']]
video_is_noise = results[results['confidence-v'] > results['confidence']]

In [14]:
print("Total counts: ", results.shape[0])
print("Textual noise counts: ", text_is_noise.shape[0])
print("Visual noise counts: ", video_is_noise.shape[0])
print("Audio noise counts: ", audio_is_noise.shape[0])

Total counts:  4654
Textual noise counts:  113
Visual noise counts:  1958
Audio noise counts:  2080
